### Цели и описание проекта для "ВИКИШОП" (2 Варианта с BERT и без BERT)



**Описание проекта:**  
Интернет-магазин «Викишоп» запускает новый сервис, который позволяет пользователям редактировать и дополнять описания товаров, как в вики-сообществах. В рамках этого сервиса клиенты могут предлагать правки, а также комментировать изменения других пользователей. Однако, с увеличением объема комментариев возникает необходимость в автоматическом выявлении токсичных, агрессивных или оскорбительных высказываний. Для этого требуется разработать инструмент для классификации комментариев, который будет определять токсичность и отправлять соответствующие сообщения на модерацию.

**Цель исследования:**  
Разработать и обучить модель машинного обучения, способную классифицировать комментарии на токсичные и нетоксичные, обеспечив значение метрики качества F1 не ниже 0.75. Использовать различные методы и подходы, включая традиционные модели машинного обучения, такие как логистическая регрессия, а также более сложные модели на основе BERT для улучшения результатов.

**Ход исследования:**

1. **Загрузка данных:**  
   Для обучения модели используется датасет, содержащий 159,929 комментариев. Столбец `text` содержит текстовые данные, а столбец `toxic` — целевую метку, где 0 — комментарий не токсичен, 1 — токсичен.

2. **Предобработка данных:**  
   - Проверка и очистка данных от дубликатов и пропусков.
   - Устранение дисбаланса классов с помощью методов андерсемплинга, чтобы привести количество нейтральных и токсичных комментариев к одинаковому значению.
   - Лемматизация текста и удаление стоп-слов для улучшения качества модели.
   - Преобразование текста в числовые векторы с использованием методов, таких как TF-IDF или модели на основе BERT.

3. **Обучение моделей:**  
   - **Модели без BERT:** Подбор гиперпараметров логистической регрессиис использованием кросс-валидации и метрики F1.
   - **Модели с BERT:** Использование модели huggingface для извлечения признаков из текстов и их классификации с помощью вышеуказанных моделей. Подбор гиперпараметров и оценка метрики F1 на тестовых данных.

4. **Оценка моделей:**  
   Оценка всех моделей с использованием метрики F1, сравнение их производительности и выбор наиболее эффективной модели для дальнейшего использования в продакшн-среде.

5. **Выводы и рекомендации:**  
   - Определение модели, которая дает наилучшие результаты для задачи классификации токсичных комментариев.
   - Рекомендации по улучшению качества классификации, возможно, с учетом дополнительной доработки предобработки данных.

### Загрузка и предобработка данных

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import torch
import spacy
import nltk
import transformers as ppb

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from pymystem3 import Mystem
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from transformers import AutoTokenizer, AutoModel

RANDOM_STATE = 42
TEST_SIZE = 0.2

In [ ]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv',index_col=0)
df.sample(5)

,text,toxic
144693,"""\n\n The two studies are prominently discussed in welfare's effect on poverty and they are """"Do social-welfare policies reduce poverty? A cross-national assessment"""" and """"Determinants of relative poverty in advanced capitalist democracies"""". """,0
123711,Termination\n\nDoes anyone know if and why Connoly was fired from the Antarctic Survey?,0
122203,"Welcome back. Here is a crapping cat, a universal symbol of relief. vzaak",1
123287,"Revert from 12 Feb 2008\nI've just reverted an edit for the same reason as given on An_Exceptionally_Simple_Theory_of_Everything#Recent_reverts_.2F_.22no_new_Einstein. In addition, this edit here appears to be a mere duplication, which should be avoided. If there would be more about Lisi the person, then it should be added here; but the new article appears more to be about the general realm of the theory, and therefore would not belong here. Thanks, Jens",0
101967,""": Hi lads, just passing by, totally agree with the above comment. I believe in KISS: """"Keep Infoboxes Simple Stupid"""" (Hmm, that'd make a good userbox..) '''''' [ contribs ] \n\n""",0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [ ]:
df.describe()

,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
df['toxic'].value_counts()

,count
toxic,
0,143106
1,16186


In [ ]:
df.isna().sum()

,0
text,0
toxic,0


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df['text'].duplicated().sum()

#### Выводы по разделу Загрузка и предобработка данных

- Данные загружены с исходным индексом.
- Всего в данных 159 929 комментарий, 16 186 негативных и 143 106 нейтральных комментариев. Требуется устранить дисбаланс классов.
- В данных не обнаружены ни полные и ни неявные дубликаты
- В данных не обнаружены пропуски

### Обучение моделей (вариант без BERT)

#### Подготовка данных

Так как данных много и чтобы код не выполнялся длительное время, принято решение отобрать случайные 30 000 объкетов и обучить на них модель, также принято решение не балансировать тренировочную выборку

In [ ]:
# выбор тренировочной и тестовой выборки
data = df.sample(30000, random_state=RANDOM_STATE)
X = data.drop(columns='toxic')
y = data['toxic']
X['text'] = X['text'].str.lower()
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

In [ ]:
# сохранение тренировочной и тестовой выборки для обучения модели с BERT
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
%%time
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
num_cores = os.cpu_count()


# функция лемматизации
def lemmatize(texts):
    lemmatized_texts = []
    for doc in nlp.pipe(texts, batch_size=100, n_process=num_cores):
        lemmatized_texts.append(" ".join([token.lemma_ for token in doc]))
    return lemmatized_texts


# функция очистки текстов
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    clean_text = " ".join(text.split())
    return clean_text

In [ ]:
%%time
X_train['text'] = X_train['text'].apply(clear_text) # очистка текста

X_train.head()

In [ ]:
%%time
X_train['lemm_text'] = lemmatize(X_train['text']) # лемматизация текста
X_train.head()

In [ ]:
%%time
# очистка и лемматизация тестовой выборки
X_test['text'] = X_test['text'].apply(clear_text) # очистка текста
X_test['lemm_text'] = lemmatize(X_test['text']) # лемматизация текста
X_test.head()

#### Обучение модели

In [ ]:
# загрузка стоп-слов
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))

##### Пайплан

In [ ]:
# создание пайплайна

pipe_final = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), stop_words=stopwords)),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid = [
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
                      random_state=RANDOM_STATE,
                      max_iter=100,
                      class_weight='balanced',
                      penalty='l2'
                  )],
        'models__C': [0.01, 0.1, 1, 10, 100],
        'models__solver': ['liblinear', 'saga', 'lbfgs'],
        'models__class_weight': ['balanced'],
        'models__penalty': ['l2']
    },

    # Добавление elasticnet регуляризации
    {
        'models': [LogisticRegression(
                      random_state=RANDOM_STATE,
                      max_iter=100,
                      class_weight='balanced',
                      solver='saga'
                  )],
        'models__C': [0.01, 0.1, 1, 10, 100],
        'models__penalty': ['elasticnet'],  # ElasticNet регуляризация
        'models__l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0],  # Соотношение l1 и l2
        'models__class_weight': ['balanced'],  # Балансировка классов
    }

    # словарь для модели RandomForestClassifier()
    {
        'models': [RandomForestClassifier(
                      random_state=RANDOM_STATE,
                      class_weight='balanced'
                  )],
        'models__n_estimators': range(50, 100),  # Количество деревьев в лесу
        'models__max_depth': range(2, 10),      # Максимальная глубина дерева
    },

    # словарь для модели GradientBoostingClassifier
    {
        'models': [GradientBoostingClassifier(
                      random_state=RANDOM_STATE
                  )],
        'models__n_estimators': range(50, 100),
        'models__learning_rate': [0.01, 0.1, 0.2],
        'models__max_depth': range(2, 5),
    },

    # словарь для модели XGBoost
    {
        'models': [XGBClassifier(
                      random_state=RANDOM_STATE,
                      use_label_encoder=False,
                      eval_metric='logloss'
                  )],
        'models__n_estimators': range(50, 100),
        'models__learning_rate': [0.01, 0.1, 0.2],
        'models__max_depth': range(2, 5),
        'models__scale_pos_weight': [1, 10, 25],  # Для дисбаланса классов
    },

]

In [ ]:
X_train = X_train['lemm_text']

In [ ]:
%%time
# поиск оптимальных параметров c помощью RandomizedSearchCV
stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

randomized_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=stratified_cv,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    n_iter=50,
    verbose=2
)


randomized_search.fit(X_train, y_train)

best_model = randomized_search.best_estimator_

print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_);
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_);

In [ ]:
result = pd.DataFrame(randomized_search.cv_results_)
result[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score').head(10)

In [ ]:
result[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score')['params'].iloc[0]

In [ ]:
y_pred = best_model.predict(X_test['lemm_text'])
f1 = f1_score(y_test, y_pred)
print ('Метрика лучшей модели на тестовой выборке:', f1);

#### Выводы по разделу обучение модели(вариант без BERT)
- устранен дисбаланс классов
- проведена очистка и лемматизаця комментариев
- создана матрица TF-IDF
- на кроссвалидации лучшей моделью оказалась LogisticRegression c параметрами
  - 'models__solver': 'lbfgs',
  - 'models__penalty': 'l2',
  - 'models__class_weight': 'balanced',
  - 'models__C': 0.1,
  - 'models': LogisticRegression(class_weight='balanced', random_state=42
  с метрикой f1 0,76
- на тестовой выборке лучшая модель показала метрику f1 0,77
- думаю можно было бы улышить метрику если обучить на всем датасете, но это требует дополнительного времени для подготовки и обучения моделей

### Обучение моделей (вариант с BERT)

#### Подготовка данных

Принято решение использовать модель 'unitary/toxic-bert'. При этом выбрать тренировочную выборку в размере 500 объектов и  протестировать на 300 объектах

In [ ]:
def prepare_text_with_toxicbert(df):
    # Загрузка модели unitary/toxic-bert и токенизатора
    pretrained_weights = 'unitary/toxic-bert'
    tokenizer = AutoTokenizer.from_pretrained(pretrained_weights, truncation=True)
    model = AutoModel.from_pretrained(pretrained_weights)

    # Токенизация текста
    max_length = 512  # Ограничение на длину токенов
    tokenized = tokenizer(
        list(df['text']),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    # Извлечение эмбеддингов из модели
    with torch.no_grad():
        outputs = model(**tokenized)

    # Используем [CLS] токен для представления текста
    features = outputs.last_hidden_state[:, 0, :].numpy()
    labels = df['toxic'].values

    return features, labels

In [ ]:
# создание батча для обучения
batch_1 = df_train.sample(500,random_state=RANDOM_STATE) # при выборке в 1000 строк colab лег, принято рещение обучить на 500 строк
batch_1['toxic'].value_counts()

In [ ]:
%%time
X_train, y_train = prepare_text_with_toxicbert(batch_1)

Проведена предобработка 500 строк комментариев с toxicbert




#### Обучение модели

In [ ]:
%%time
# подбор гиперпараметров с помощью RandomizedSearchCV
pipe_final = Pipeline([
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

randomized_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    n_iter=20,
    verbose=3
)


randomized_search.fit(X_train, y_train)

best_model = randomized_search.best_estimator_

print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_);
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_);

In [ ]:
result = pd.DataFrame(randomized_search.cv_results_)
result[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score').head(10)

In [ ]:
batch_1 = df_test.sample(300,random_state=RANDOM_STATE)
batch_1['toxic'].value_counts()

In [ ]:
%%time
X_test, y_test = prepare_text_with_toxicbert(batch_1)

In [ ]:
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
print ('Метрика лучшей модели на тестовой выборке:', f1);

#### Выводы по разделу Обучение моделей(вариант с BERT)
- Выбраны 500 случайный комментариев из датасета
- Проведена предобработка данных комментариев специальной моделю huggingface models для обработки токсичных комментариев
- Подобраны гиперпараметры для LogisticRegression. Параметры лучшей модели LogisticRegression(C=0.1, class_weight='balanced', random_state=42) с метрикой f1  равной 0,91
- на тестовой выбрке лучшая модель показала метрику F1 равной 0,95.
- при этом если не ограничиваться только 500 объектами, то можно получить еще лучшую точность для данной модели, но это требует дополнительных ресурсов и времени для предобработки



### Выводы по проекту



1. **Загрузка и предобработка данных:**  
   - Данные были успешно загружены из файла и предварительно обработаны.
   - В исходном датасете не было обнаружено ни пропусков, ни явных дубликатов, что упрощает дальнейшую работу с ними.
   - Обнаружен дисбаланс классов (16,186 токсичных и 143,106 нейтральных комментариев)

2. **Модели без использования BERT:**  
   - Для классификации комментариев использовалась модель машинного обучения,  **Logistic Regression**.
   - Были проведены следующие шаги:
     - Очистка текста, включая лемматизацию и удаление стоп-слов.
     - Преобразование текста в числовые признаки с помощью **TF-IDF**.
   - **Logistic Regression**  показала наилучший результат на кросс-валидации с метрикой F1 **0,76**, что подтверждает её высокую эффективность при данной задаче.
   - На тестовой выборке эта же модель достигла метрики F1 **0,77**, что свидетельствует о её хорошем обобщении на новых данных.

3. **Модели с использованием BERT:**  
   - Для улучшения качества классификации был выбран подход с использованием модели **huggingface models** для обработки токсичных комментариев, которая позволяет извлекать контекстуальные признаки из текстов.
   - Были выбраны 500 случайных комментариев из датасета для обучения модели на основе **huggingfac**.
   - Для классификации комментариев использовалась модель машинного обучения,  **Logistic Regression** после извлечения признаков из **huggingface models**.
   - Полученная метрика F1 для модели с использованием BERT составила **0,91**, что значительно выше, чем у традиционной модели.
   - На тестовых данных метрика F1 составила **0,95**, что подтверждает способность модели BERT классифицировать токсичные комментарии с хорошей точностью.

4. **Общий вывод:**  
   - Традиционные модели машинного обучения (**Logistic Regression**) продемонстрировали очень хорошие результаты и способны решать задачу с высокой точностью, при этом они требуют меньше вычислительных ресурсов и быстрее обучаются.
   - Использование **huggingface** позволяет улучшить качество классификации, особенно для более сложных и длинных текстов, но требует больших вычислительных мощностей и времени на обучение.
   - подход с **BERT** показывает лучшие результаты, особенно при использовании более мощных вычислительных ресурсов.

5. **Рекомендации:**  
   - Если требуется максимальная производительность при ограниченных вычислительных ресурсах, предпочтительнее использовать традиционные модели машинного обучения, такие как **Logistic Regression** с **TF-IDF**.
   - Для достижения ещё более высоких показателей точности при доступности более мощных вычислительных ресурсов можно применять подходы на основе трансформеров, таких как **huggingface**, с использованием дополнительных техник дообучения и оптимизации.

